# Testing wrap_openai + anthropic

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# P1

Citations don't render:

Anthropic docs: https://docs.claude.com/en/docs/build-with-claude/citations


LangSmith trace: https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%22feedback_stats%22%3Afalse%2C%22reference_example%22%3Afalse%2C%22start_time%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22error%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Afalse%2C%22last_queued_at%22%3Afalse%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Afalse%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22reference_example_id%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&peek=1394ca90-7be0-4061-9d8b-04a73b1726e2&peeked_trace=1394ca90-7be0-4061-9d8b-04a73b1726e2

In [ ]:
from langsmith import traceable

#ignore
inputs2 = {"messages": [ {
  "role": "tool",
  "tool_call_id": "call_1",
  "content": "{\"result\":36}",
}]}


outputs4 = {
  "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "document",
            "source": {
              "type": "text",
              #"media_type": "text/plain", need to comment out or we see a warning  https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%22feedback_stats%22%3Afalse%2C%22reference_example%22%3Afalse%2C%22start_time%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22error%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Afalse%2C%22last_queued_at%22%3Afalse%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Afalse%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22reference_example_id%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&peek=69185654-cd05-4ee9-8a72-5363a0b8ea0d&peeked_trace=69185654-cd05-4ee9-8a72-5363a0b8ea0d
              "data": "The grass is green. The sky is blue."
            },
            "title": "My Document",
            "context": "This is a trustworthy document.",
            "citations": {"enabled": True}
          },
          {
            "type": "text",
            "text": "What color is the grass and sky?"
          }
        ]
      }
    ]
}

@traceable(run_type="llm")
def llm_raw(inputs):
    return outputs4


llm_raw(inputs2)